# Correos electronicos SPAM: Un enfoque con Procesamiento de Lenguaje Natural

## Armando Misael Miranda Hernandez

Los correos electronicos no deseados en su bandeja de entreda son molestos ya que perturban la rutina del usuario. Es por eso que las cuentas de correo electronico ya tiene un filtro spam. Dado que es una de las aplicaciones del PLN mas utilizadas vamos a ver como se desarrollo un filtro de spam simple para correos electronicos.

In [1]:
# Importamos las famosas librerias
from functools import reduce

import nltk
from nltk.stem import WordNetLemmatizer
import pandas as pd
import string
import re

In [2]:
#instertamos los datos
full_corpus=pd.read_csv('SMSSpamCollection.tsv',sep='\t', header=None, names=['label','msg_body'])

#Separando los datos en ham y spam
ham_text=[]
spam_text=[]

## Bigrams

Los N-gramos se usan para modelar el lenguaje en funcion de la prediccion de palabras, es decir, predice la siguiente palabra de una oracion de palabras N-1 anteriores. Biagrams es la secuencia de 2 de N-gramos que predice la siguiente palabra de una oracion usando la palabra anterior. En lugar de considerar la historia completa de una oracion o una secuencia de palabras en particular, un modelo como biagram puede ser ocupado en terminos de una aproximacion de la historia al ocupar una historia limitada.

La identificacion de un mensaje como 'ham' o 'spam' es una tarea de clasificacion ya que la variable de destino tiene valores discretos que son "ham" o "spam". En esta practica, se usa el modelo biagram, aunque existen muchas tecnicas avanzadas que se pueden utilizar para este proposito. Para utilizar el modelo biagram para asignar un mensaje dado como "spam" o "ham", hay variospasos que deben lograrse:

#### 1. Inspeccion y separacion de mensajes en las categorias "Ham y Spam"
Inicialmente, el conjunto de datos debe inspeccionarse para ocuparlo y abordarlo para lograr la tarea. El formato de los datos dados, la cantidad de datos proporcionados la naturaleza de los datos se incluyen en esta inspeccion para indetificar la mejor aproximacion posible para la tarea.

El corpus de mensajes dado ha marcado cada mensaje como ham o spam. Ademas, hay 5568 mensajes en un DataFrame escrito en ingles que no son objetos nulos. Por lo tanto, el archivo tsv se puede leer usando DataFrame en python para clasificar esos mensajes de acuerdo con el indocador dado.

In [3]:
def separete_msg():
    for index, column in full_corpus.iterrows():
        label=column[0]
        message_text=column[1]
        if label=='ham':
            ham_text.append(message_text)
        elif label=='spam':
            spam_text.append(message_text)
separete_msg()

### 2. Preprocesamiento de texto
El preprocesamiento es la tarea de realizar los pasos de preparacion e el corpus de texto sin formato para completar de manera eficiente una extraccion de texto o procesamiento de lenguaje natural o cualquier otra tarea que incluya texto sin formato. El preprocesamiento de texto consta de varios pasos, aunque algunos de ellos pueden no aplicarse a una tarea en particular debido a la naturaleza del conjunto de datos disponible.

En esta tarea, el preprocesamiento de texto inlcuye los siguientes pasos de acuerdo con el conjunto de datos.

#### Eliminacion de signos de puntuacion

In [4]:
#Eliminacion de los signos de puntuacion de los mensajes de correo electronico
def remove_msg_punctuations(email_msg):
    puntuaction_remove_msg="".join([word for word in email_msg if word not in string.punctuation])
    return puntuaction_remove_msg

#### Convertir a minusculas:
Convertir a minusculas: La conversion de todos los caracteres del texto en un contexto comun, como lo soportes en minusculas, impide identificar dos palabras de manera diferente donde una esta en minisculas y la otra no. Por ejemplo, "Primero" y "primero" deben identificarse como iguales, por lo tanto, poner en minusculas todos los caracteres facilita la tarea. Ademas, las palabras de detencion tambien estan en minusculas, por lo que esto tammbien haria posible eliminar palabras de detencion mas adelante.

#### Tokenizing
Tokenizing: La tokenizacion es la tarea de dividir el texto en partes significativas, es decir, tokens que incluye oraciones y palabras. Un token se puede considerar como una instancia de una secuancia de caracteres en un texto particular que se agrupan para proporcionar una unidad semantica util para su porterior procesamiento. En esta tarea, la tokenizacion de palabras se realiza combinando espacios en blanco entre palabras como delimitaor. Esto se logra en Python usando expresiones regulares para dividir una cadena en subcadenas con la funcion split(), que es un tokenizador basico.

In [5]:
# Convierte el texto en minusculas y tokenizing de palabras
def tokenize_into_words(text):
    tokens=re.split('\W', text)
    return tokens

#### Palabras Lematizantes:
La derivacion es el proceso de elimar afijos(sufijos, prefijos, infijos, circunfijos) de una palabra para obtener su raiz de palabra. Aunque la lematizacion esta relacionada con la derivacion, difiere ya que la lematizacion puede capturar formas canonicas basadas en el lema de una palabra. La lematizacion ocupa un vocabulario y un analisis morfologico de las palabras que lo hacen mas rapido y preciso que la derivacion. WordNetLemmatizer ha logrado la lematizacion en lenguaje Python

In [6]:
#Lemmatizing
word_lemmatizer=WordNetLemmatizer()
def lemmatization(tokenized_word):
    lemmatized_text=[word_lemmatizer.lemmatize(word)for word in tokenized_word]
    return ' '.join(lemmatized_text)

def preprocessing_msgs(corpus):
    categorized_text=pd.DataFrame(corpus)
    categorized_text['non_punc_message_body']=categorized_text[0].apply(lambda msg: remove_msg_punctuations(msg))
    categorized_text['tokenized_msg_body']=categorized_text['non_punc_message_body'].apply(lambda msg: tokenize_into_words(msg.lower()))
    categorized_text['lemmatized_msg_words']=categorized_text['tokenized_msg_body'].apply(lambda word_list: lemmatization(word_list))
    return categorized_text['lemmatized_msg_words']

### 3. Extracción de características
Después de la etapa de procesamiento, las caracteristicas deben extraerse del texto. Las caracteristicas son las unidades que admiten la tarea de clasificación, y las bigrams son las caracteristicas en esta tarea de clasificación de mensajes. Los birams o las caracteristicas se extraen dek texto preprocesado Inicialmente, los unigramas se adquieren, y luego esos unigramas se utilizan para obtener los unigramas en cada corpus ("ham" y "spam").

In [7]:
# Extraccion de caracteristicas. Ejemplos: n-grass
def feature_extraction(preprocessed_text):
    bigrams=[]
    unigram_lists=[]
    for msg in preprocessed_text:
        # Agregando end of and start of a mensaje
        msg='<s>' + msg + '</s>'
        unigram_lists.append(msg.split())
    unigrams=[uni_list for sub_list in unigram_lists for uni_list in sub_list]
    bigrams.extend(nltk.bigrams(unigrams))
    return bigrams

### 4. Eliminacion de Stop Words

Hay ciertas parabras en un idioma (se utiliza ingles en la practica) que son para necerias para una oracion o una secuencia de palabras, auque no contribuyen al significado

En lugar de eliminar las palabras de detención en el paso de preprocesamiento, se realiza después de extraer las características del corpus para evitar la
ausencia de bigrams con palabras de una parada (('use', ’your'), ('to', ’win’ )) al adquirir las funciones, ya que tienen un impacto en el resultado final de la aplicación. Las palabras de detención se pueden ignorar en esta Recuperación de información orientada a palabras clave debido a su efecto en la precisión
de la recuperación.


In [8]:
# Eliminando biagrams solo con stop words
stopwords=nltk.corpus.stopwords.words('english')
def filter_stopwords_biagrams(biagram_list):
    filtered_biagrams=[]
    for biagram in biagram_list:
        if biagram[0] in stopwords and biagram[1] in stopwords:
            continue
        filtered_biagrams.append(biagram)
    return filtered_biagrams

### 5. Obtener distribución de frecuencia de características

La distribución de frecuencia se utiliza para obtener la frecuencia de aparición de cada elemento de vocabulario en un texto determinado.

In [9]:
#Adquiriendo la frecuencia de caractericticas
def ham_bigram_feature_frequency():
    # Frecuencia de caracteristicas para mensajes ham
    ham_biagrams=feature_extraction(preprocessing_msgs(ham_text))
    ham_biagrams_frecuency=nltk.FreqDist(filter_stopwords_biagrams(ham_biagrams))
    return ham_biagrams_frecuency

def spam_bigram_feature_frequency():
    # Frecuencia de caractericticas para mensajes spam
    spam_bigrams=feature_extraction(preprocessing_msgs(spam_text))
    spam_bigrams_frecuency=nltk.FreqDist(filter_stopwords_biagrams(spam_bigrams))
    return spam_bigrams_frecuency

### 6. Construyendo un modelo para la predicción
El modelo para clasificar un mensaje dado como "ham" o "spam" se ha abordado calculando las probabilidades de bigram dentro de cada corpus.
Inicialmente, el mensaje dado debe procesarse previamente para avanzar con la clasificación, incluida la eliminación de signos de puntuación, el cambio de
todos los caracteres a minúsculas, la tokenización y la lematización. Luego, los bigrams se extraen del texto preprocesado para calcular finalmente la
probabilidad de que el texto esté en cada corpus "ham" o "spam".


In [10]:
# Calculando probabilidades del biagram
def biagram_probability(message):
    probability_h=1
    probability_s=1
    # Preprocesando los mensajes de entrada
    punc_removed_message="".join(word for word in message if word not in string.punctuation)
    punc_removed_message='<s>'+punc_removed_message+'</s>'
    tokenized_msg=re.split('\s+', punc_removed_message)
    lemmatized_msg=[word]

In [12]:
# Calculando probabilidades del bigram
def bigram_probability(message):
    probability_h = 1
    probability_s = 1
    # Preprocesando los mensaje de entrada
    punc_removed_message = "".join(word for word in message if word not in string.punctuation)
    punc_removed_message = '<s> ' +punc_removed_message +' </s>'
    tokenized_msg = re.split('\s+', punc_removed_message)
    lemmatized_msg = [word_lemmatizer.lemmatize(word)for word in tokenized_msg]
    # bigrams para el mensaje
    bigrams_for_msg = list(nltk.bigrams(lemmatized_msg))
    # Eliminamos stop words
    ham_unigrams = [word for word in feature_extraction(preprocessing_msgs(ham_text)) if word not in stopwords]
    spam_unigrams = [word for word in feature_extraction(preprocessing_msgs(spam_text)) if word not in stopwords]
    # Frecuencias de bigrams extraidas
    ham_frequency = ham_bigram_feature_frequency()
    spam_frequency  = spam_bigram_feature_frequency()
    print('========================== Calculando Probabilidades ==========================')
    print('----------- Frecuencias Ham ------------')
    for bigram in bigrams_for_msg:
        # probabilidad de la primera palabra en bigram
        ham_probability_denominator = 0
        # probabilidad de bigram (suavizado) 
        ham_probability_of_bigram = ham_frequency[bigram] + 1
        print(bigram, ' ocurre ', ham_probability_of_bigram)
        for (first_unigram, second_unigram) in filter_stopwords_biagrams(ham_unigrams):
            ham_probability_denominator += 1
            if(first_unigram == bigram[0]):
                ham_probability_denominator += ham_frequency[first_unigram, second_unigram]
        probability = ham_probability_of_bigram / ham_probability_denominator
        probability_h *= probability
        print(probability_h )
    print('\n')
    print('----------- Frecuencias Spam ------------')
    for bigram in bigrams_for_msg:
        # probabilidad de la primera palabra en bigram
        spam_probability_denominator = 0
        # probabilidad de bigram (suavizado) 
        spam_probability_of_bigram = spam_frequency[bigram] + 1
        print(bigram, ' ocurre ', spam_probability_of_bigram)
        for (first_unigram, second_unigram) in filter_stopwords_biagrams(spam_unigrams):
            spam_probability_denominator += 1
            if(first_unigram == bigram[0]):
                spam_probability_denominator += spam_frequency[first_unigram, second_unigram]
        probability = spam_probability_of_bigram / spam_probability_denominator
        probability_s *= probability
        print(probability_s )
    print('\n')
    print('Probabilidad Ham: ' +str(probability_h))
    print('Probabildiad Spam: ' +str(probability_s))
    print('\n')
    if(probability_h >= probability_s):
        print('\"' +message +'\" es un mensaje Ham')
    else:
        print('\"' +message +'\" es un mensaje Spam')
    print('\n')
bigram_probability('Click here,  ..to win an iphone 11 pro max')
bigram_probability('Homework')

========================== Calculando Probabilidades ==========================
----------- Frecuencias Ham ------------
('<s>', 'Click')  ocurre  1
1.675126053235506e-05
('Click', 'here')  ocurre  1
2.8085878531185657e-10
('here', 'to')  ocurre  1
4.706315419874601e-15
('to', 'win')  ocurre  2
1.2633725490912166e-19
('win', 'an')  ocurre  1
2.1168756372902877e-24
('an', 'iphone')  ocurre  1
3.527890869425851e-29
('iphone', '11')  ocurre  1
5.915012439726122e-34
('11', 'pro')  ocurre  1
9.916863561221409e-39
('pro', 'max')  ocurre  1
1.662675803303167e-43
('max', '</s>')  ocurre  1
2.787666493365916e-48


----------- Frecuencias Spam ------------
('<s>', 'Click')  ocurre  1
6.109855196431844e-05
('Click', 'here')  ocurre  1
3.733261148986829e-09
('here', 'to')  ocurre  1
2.2806898093877625e-13
('to', 'win')  ocurre  19
1.8728112359913339e-16
('win', 'an')  ocurre  4
4.46598601643337e-20
('an', 'iphone')  ocurre  1
2.716040878448805e-24
('iphone', '11')  ocurre  1
1.65956304439008e-28
(